In [1]:
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [2]:
filedate = '20220428'

In [3]:
url = 'https://www.rakuten-sec.co.jp/web/market/data/list.html'

In [4]:
driver = webdriver.Chrome("/Users/ricew/anaconda3/pkgs/chromedriver-binary-2.38-0/Library/bin/chromedriver")

In [5]:
driver.get(url)
time.sleep(3)

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
# データを取得
c = driver.page_source
# HTMLをもとに、オブジェクトを作る
soup = BeautifulSoup(c)

In [8]:
# 目的の部分を切り出します。
# summary = soup.find("div",{'id':'tFut'})
summary = soup.find("div",{'class':'indx1_1'})

# tableを見つけます。
tables = summary.find_all('table')

In [9]:
# データを格納するためのリストです。
data = []

# テーブルから行をすべて探し出します。
rows = tables[0].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
現在値
前日比
前日比率
更新日時


日経225
26,847.90
+461.27
1.75%
04/28


TOPIX
1,899.62
+38.86
2.09%
04/28


JPX日経400
17,122.20
+340.56
2.03%
04/28


東証プライム市場指数
977.08
+20.04
2.09%
04/28 16:00


東証スタンダード市場指数
965.34
+9.75
1.02%
04/28 16:00


東証グロース市場指数
878.94
-7.13
-0.80%
04/28 16:00


旧東証市場第一部指数
1,899.62
+38.86
2.09%
04/28 16:00


東証グロース市場Core指数
844.80
-17.64
-2.05%
04/28


マザーズ総合指数
694.41
-6.04
-0.86%
04/28


日経平均VI指数
25.07
-1.54
-5.79%
04/28


東証REIT指数
1,975.75
-21.91
-1.10%
04/28


NYダウ
32,977.21
-939.18
-2.77%
04/29 16:59


NASDAQ
12,334.64
-536.89
-4.17%
04/29 17:15


S&P500指数
4,131.93
-155.57
-3.63%
04/29 16:59


英FT100指数
7,544.55
+35.36
0.47%
04/29


仏CAC40指数
6,533.77
+25.63
0.39%
04/29


独DAX30指数
14,097.88
+118.04
0.84%
04/29


ロシアRTS指数
1,081.52
+39.61
3.80%
04/29


ブラジルボベスパ指数
107,876.16
-2042.81
-1.86%
04/29 17:22


インドSENSEX
57,060.87
-460.19
-0.80%
04/29


香港ハンセン指数
21,089.39
+813.22
4.01%
04/29


ハンセンH株指数
7,298.69
+380.07
5.49%
04/29


ハンセンレッドチップ指数
3,926.53
+59.32
1.53%
04/29


上海総合指数
3

In [10]:
index_name = []
current_val = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0
# 行を保持する変数を用意します。

# pdfの文字列があるcellを見つけだします。
for item in data:
    if  i %6 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %6 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %6 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %6 == 3:
        diff_ratio.append(item) 
        i = i + 1
    elif i %6 == 4:
        update.append(item) 
        i = i + 1
    elif i %6 == 5:
        dells.append(item) 
        i = i + 1

In [11]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [12]:
# 連結してDataFrameにします。
df_market1 = pd.concat([index_name,current_val,diff_val,diff_ratio,update],axis=1)

In [13]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market1.columns = df_market1.iloc[0]
df_market1.drop(df_market1.index[0],inplace=True)

In [14]:
df_market1

,指標,現在値,前日比,前日比率,更新日時
1,日経225,"26,847.90",+461.27,1.75%,04/28
2,TOPIX,"1,899.62",+38.86,2.09%,04/28
3,JPX日経400,"17,122.20",+340.56,2.03%,04/28
4,東証プライム市場指数,977.08,+20.04,2.09%,04/28 16:00
5,東証スタンダード市場指数,965.34,+9.75,1.02%,04/28 16:00
6,東証グロース市場指数,878.94,-7.13,-0.80%,04/28 16:00
7,旧東証市場第一部指数,"1,899.62",+38.86,2.09%,04/28 16:00
8,東証グロース市場Core指数,844.80,-17.64,-2.05%,04/28
9,マザーズ総合指数,694.41,-6.04,-0.86%,04/28
10,日経平均VI指数,25.07,-1.54,-5.79%,04/28


In [15]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[1].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
現在値
前日比
前日比率
更新日時


日経225先物（期近）
27,210
+340
1.27%
04/29


CME日経平均先物
26,730
-545
-2.00%
04/29 16:37


SGX日経平均先物
26,755.00
-625.00
-2.28%
04/30 05:13


東証マザース先物（期近）
702.0
+9.0
1.30%
04/29


JPX日経400先物（期近）
17,395
+235
1.37%
04/29


JPX国債先物
11,737.97
+24.32
0.21%
04/28


JPX国債先物レバレッジ
13,696.31
+56.64
0.42%
04/28


JPX国債先物インバース
8,468.84
-17.62
-0.21%
04/28


JPX国債先物ダブルインバース
7,129.59
-29.73
-0.42%
04/28


台湾加権先物（期近）
0
--
--%
--


FTSE中国50先物（期近）
0
--
--%
--


E-mini S&P 500
4,127.5
-156.0
-3.64%
04/29 16:37


E-mini S&P Mid Cap 400
2,495.1
-69.8
-2.72%
04/29 16:37


E-mini S&P Small Cap 600
1,214.3
-32.8
-2.63%
04/29 16:38


E-mini NASDAQ-100
12,852
-602.75
-4.48%
04/29 16:38


E-mini Dow
32,882
-946
-2.80%
04/29 16:37


E-mini Nikkei225
26,730
-550
-2.02%
04/29 16:38


SGX Nifty 50 Index Futures
16,960.00
-166.50
-0.97%
04/30 05:14


In [16]:
index_name = []
current_val = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %6 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %6 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %6 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %6 == 3:
        diff_ratio.append(item) 
        i = i + 1
    elif i %6 == 4:
        update.append(item) 
        i = i + 1
    elif i %6 == 5:
        dells.append(item) 
        i = i + 1

In [17]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [18]:
# 連結してDataFrameにします。
df_market2 = pd.concat([index_name,current_val,diff_val,diff_ratio,update],axis=1)

In [19]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market2.columns = df_market2.iloc[0]
df_market2.drop(df_market2.index[0],inplace=True)

In [20]:
df_market2

,指標,現在値,前日比,前日比率,更新日時
1,日経225先物（期近）,"27,210",+340,1.27%,04/29
2,CME日経平均先物,"26,730",-545,-2.00%,04/29 16:37
3,SGX日経平均先物,"26,755.00",-625.00,-2.28%,04/30 05:13
4,東証マザース先物（期近）,702.0,+9.0,1.30%,04/29
5,JPX日経400先物（期近）,"17,395",+235,1.37%,04/29
6,JPX国債先物,"11,737.97",+24.32,0.21%,04/28
7,JPX国債先物レバレッジ,"13,696.31",+56.64,0.42%,04/28
8,JPX国債先物インバース,"8,468.84",-17.62,-0.21%,04/28
9,JPX国債先物ダブルインバース,"7,129.59",-29.73,-0.42%,04/28
10,台湾加権先物（期近）,0,--,--%,--


In [21]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[2].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
買い
売り
前日比（買い）
前日比率（買い）
更新日時


米ドル/円
129.75
129.78
0.00
0.00%
04/30 05:59


ユーロ/円
136.84
136.92
-0.01
-0.01%
04/30 06:06


豪ドル/円
91.70
91.75
0.00
0.00%
04/30 05:59


NZドル/円
83.82
83.89
0.00
0.00%
04/30 06:00


カナダドル/円
100.96
101.01
0.00
0.00%
04/30 05:59


英ポンド/円
163.20
163.31
-0.01
-0.01%
04/30 06:06


スイスフラン/円
133.34
133.41
0.00
0.00%
04/30 05:59


香港ドル/円
16.5456
16.5498
0.0000
0.00%
04/30 05:59


トルコリラ/円
8.7358
8.7497
0.0000
0.00%
04/30 05:59


南アランド/円
8.2052
8.2149
0.0000
0.00%
04/30 06:00


ロシアルーブル/円
1.6971
1.8499
0.0000
0.00%
04/30 05:59


韓国ウォン/円
0.1027
0.1028
0.0000
0.00%
04/30 05:59


メキシコペソ/円
6.3515
6.3623
0.0000
0.00%
04/30 05:59


ブラジルレアル/円
26.107
26.118
0.000
0.00%
04/30 06:00


インドルピー/円
1.6960
1.6971
0.0000
0.00%
04/30 05:59


中国人民元/円
19.6444
19.6519
0.0000
0.00%
04/30 05:59


シンガポールドル/円
93.8078
93.8634
0.0000
0.00%
04/30 06:00


タイバーツ/円
3.7896
3.7926
0.0000
0.00%
04/30 06:00


マレーシアリンギット/円
29.8117
29.8392
0.0000
0.00%
04/30 05:59


インドネシアルピア/円
0.8954
0.8959
0.0000
0.00

In [22]:
index_name = []
current_buy = []
current_sel = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i % 7 == 0:
        index_name.append(item)
        i = i + 1
    elif i % 7 == 1:
        current_buy.append(item) 
        i = i + 1
    elif i % 7 == 2:
        current_sel.append(item) 
        i = i + 1
    elif i % 7 == 3:
        diff_val.append(item) 
        i = i + 1
    elif i % 7 == 4:
        diff_ratio.append(item) 
        i = i + 1
    elif i % 7 == 5:
        update.append(item) 
        i = i + 1
    elif i % 7 == 6:
        dells.append(item) 
        i = i + 1

In [23]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_buy = Series(current_buy)
current_sel = Series(current_sel)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [24]:
# 連結してDataFrameにします。
df_market3 = pd.concat([index_name,current_buy,current_sel,diff_val,diff_ratio,update],axis=1)

In [25]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market3.columns = df_market3.iloc[0]
df_market3.drop(df_market3.index[0],inplace=True)

In [26]:
df_market3

,指標,買い,売り,前日比（買い）,前日比率（買い）,更新日時
1,米ドル/円,129.75,129.78,0.00,0.00%,04/30 05:59
2,ユーロ/円,136.84,136.92,-0.01,-0.01%,04/30 06:06
3,豪ドル/円,91.70,91.75,0.00,0.00%,04/30 05:59
4,NZドル/円,83.82,83.89,0.00,0.00%,04/30 06:00
5,カナダドル/円,100.96,101.01,0.00,0.00%,04/30 05:59
6,英ポンド/円,163.20,163.31,-0.01,-0.01%,04/30 06:06
7,スイスフラン/円,133.34,133.41,0.00,0.00%,04/30 05:59
8,香港ドル/円,16.5456,16.5498,0.0000,0.00%,04/30 05:59
9,トルコリラ/円,8.7358,8.7497,0.0000,0.00%,04/30 05:59
10,南アランド/円,8.2052,8.2149,0.0000,0.00%,04/30 06:00


In [27]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[3].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
現在値
前日比
前日比率
更新日時


原油（WTI原油先物）
104.11
-1.25
-1.19%
04/29 16:59


天然ガス（Henry Hub）先物（NYMEX）
7.275
+0.385
5.59%
04/29 18:54


金（現物 1oz.あたり）
1,896.3959
+2.0536
0.11%
04/29 17:00


Gold先物（COMEX)
1,906
+17.3
0.92%
04/29 16:37


プラチナ先物（NYMEX）
937.6
+11.7
1.29%
04/29 18:54


Silver先物（COMEX)
22.735
-0.394
-1.70%
04/29 16:37


Copper先物（COMEX)
4.378
-0.0355
-0.80%
04/29 16:37


Corn先物（CBOT）
818.25
+2.25
0.28%
04/29 16:37


Wheat先物（CBOT）
1,044
-30
-2.79%
04/29 16:37


Soybeans先物（CBOT）
1,713
+6.5
0.38%
04/29 16:37


In [28]:
index_name = []
current_val = []
diff_val = []
diff_ratio = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %6 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %6 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %6 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %6 == 3:
        diff_ratio.append(item) 
        i = i + 1
    elif i %6 == 4:
        update.append(item) 
        i = i + 1
    elif i %6 == 5:
        dells.append(item) 
        i = i + 1

In [29]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
diff_ratio = Series(diff_ratio)
update = Series(update)

In [30]:
# 連結してDataFrameにします。
df_market4 = pd.concat([index_name,current_val,diff_val,diff_ratio,update],axis=1)

In [31]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market4.columns = df_market4.iloc[0]
df_market4.drop(df_market4.index[0],inplace=True)

In [32]:
df_market4

,指標,現在値,前日比,前日比率,更新日時
1,原油（WTI原油先物）,104.11,-1.25,-1.19%,04/29 16:59
2,天然ガス（Henry Hub）先物（NYMEX）,7.275,+0.385,5.59%,04/29 18:54
3,金（現物 1oz.あたり）,"1,896.3959",+2.0536,0.11%,04/29 17:00
4,Gold先物（COMEX),"1,906",+17.3,0.92%,04/29 16:37
5,プラチナ先物（NYMEX）,937.6,+11.7,1.29%,04/29 18:54
6,Silver先物（COMEX),22.735,-0.394,-1.70%,04/29 16:37
7,Copper先物（COMEX),4.378,-0.0355,-0.80%,04/29 16:37
8,Corn先物（CBOT）,818.25,+2.25,0.28%,04/29 16:37
9,Wheat先物（CBOT）,"1,044",-30,-2.79%,04/29 16:37
10,Soybeans先物（CBOT）,"1,713",+6.5,0.38%,04/29 16:37


In [33]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[4].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
年利回り
前日比
更新日時


日本国債3年
-0.038
+0.003
04/29


日本国債5年
0.004
0.000
04/29


日本国債10年
0.220
+0.003
04/29


米国3年国債
2.894
0.000
04/30 06:05


米国10年国債
2.938
0.000
04/30 06:05


ドイツ10年国債
0.936
-0.002
04/30 04:59


イギリス10年国債
1.910
+0.003
04/30 01:59


フランス10年国債
1.450
+0.002
04/30 04:59


スペイン10年国債
1.998
+0.094
04/30 04:59


ポルトガル10年国債
2.025
-0.007
04/30 02:00


ユーロ圏10年国債
0.936
-0.002
04/30 04:59


イタリア10年国債
2.778
-0.002
04/30 04:00


ギリシャ10年国債
3.364
+0.029
04/30 06:46


ハンガリー10年国債
7.010
+0.170
04/29


In [34]:
index_name = []
current_val = []
diff_val = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %5 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %5 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %5 == 2:
        diff_val.append(item) 
        i = i + 1
    elif i %5 == 3:
        update.append(item) 
        i = i + 1
    elif i %5 == 4:
        dells.append(item) 
        i = i + 1

In [35]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
diff_val = Series(diff_val)
update = Series(update)

In [36]:
# 連結してDataFrameにします。
df_market5 = pd.concat([index_name,current_val,diff_val,update],axis=1)

In [37]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market5.columns = df_market5.iloc[0]
df_market5.drop(df_market5.index[0],inplace=True)

In [38]:
df_market5

,指標,年利回り,前日比,更新日時
1,日本国債3年,-0.038,+0.003,04/29
2,日本国債5年,0.004,0.000,04/29
3,日本国債10年,0.220,+0.003,04/29
4,米国3年国債,2.894,0.000,04/30 06:05
5,米国10年国債,2.938,0.000,04/30 06:05
6,ドイツ10年国債,0.936,-0.002,04/30 04:59
7,イギリス10年国債,1.910,+0.003,04/30 01:59
8,フランス10年国債,1.450,+0.002,04/30 04:59
9,スペイン10年国債,1.998,+0.094,04/30 04:59
10,ポルトガル10年国債,2.025,-0.007,04/30 02:00


In [39]:
# データを格納するためのリストです。
data = []
rows = []
# テーブルから行をすべて探し出します。
rows = tables[5].find_all('tr')

# 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
# try～exceptを入れることにより、その間でタグが見つからなくてエラーになっても続ける
for tr in rows:
    try:
        cols = tr.find_all('th')
        # textを探し出します。
        for th in cols:
            text = th.find(text=True) 
            print(text)
            data.append(text)
        for th in cols:
            text = th.find("a").get_text()
            print(text)
            data.append(text)
        cols = tr.find_all('td')
        # textを探し出します。
        for td in cols:
            text = td.find(text=True) 
            print(text)
            data.append(text) 
    except:
        continue

指標
政策金利
更新日時


日本 無担保コール翌日物
-0.021
2022/04/28


日本 公定歩合
0.3
2008/12/19


アメリカ フェデラルファンド金利
0.50
2022/03/16


カナダ 翌日物金利誘導目標
1.00
2022/04/13


ユーロ 市場調整金利
--
2016/03/10


イギリス レポ金利
0.75
2022/03/17


スイス 3ヵ月物LIBOR誘導目標
-0.7500
2022/03/24


南アフリカ レポ金利
4.25
2022/03/24


香港 再割引金利
5
2022/04/26


オーストラリア　政策金利
0.10
2020/11/04


ニュージーランド 目標短期金利
1.50
2022/04/29


インド レポ金利
4.00
2022/03/02


中国 貸出金利
4.35
2015/10/24


ブラジル Selicターゲットレート
11.75
2022/03/16


ロシア リファイナンス金利
--
--


トルコ 中央銀行ON DEPO
12.50
2022/04/29


メキシコ オーバーナイト金利
6.61
2022/04/29 17:15


In [40]:
index_name = []
current_val = []
update = []
dells = []
i = 0

# それぞれの列に値を割り振ります。
for item in data:
    if   i %4 == 0:
        index_name.append(item) 
        i = i + 1
    elif i %4 == 1:
        current_val.append(item) 
        i = i + 1
    elif i %4 == 2:
        update.append(item) 
        i = i + 1
    elif i %4 == 3:
        dells.append(item) 
        i = i + 1

In [41]:
# それぞれをSeriesにする
index_name = Series(index_name)
current_val = Series(current_val)
update = Series(update)

In [42]:
# 連結してDataFrameにします。
df_market6 = pd.concat([index_name,current_val,update],axis=1)

In [43]:
# 0行名をカラム名として指定し、カラム名と指定した行を削除
df_market6.columns = df_market6.iloc[0]
df_market6.drop(df_market6.index[0],inplace=True)

In [44]:
df_market6

,指標,政策金利,更新日時
1,日本 無担保コール翌日物,-0.021,2022/04/28
2,日本 公定歩合,0.3,2008/12/19
3,アメリカ フェデラルファンド金利,0.50,2022/03/16
4,カナダ 翌日物金利誘導目標,1.00,2022/04/13
5,ユーロ 市場調整金利,--,2016/03/10
6,イギリス レポ金利,0.75,2022/03/17
7,スイス 3ヵ月物LIBOR誘導目標,-0.7500,2022/03/24
8,南アフリカ レポ金利,4.25,2022/03/24
9,香港 再割引金利,5,2022/04/26
10,オーストラリア 政策金利,0.10,2020/11/04


In [45]:
# 行列入れ替え
df_market11 = df_market1.T

In [46]:
# 1行目だけ残して後は削除
df_market12 = df_market11.drop(df_market11.index[[2,3,4]]).reset_index(drop=True)

In [47]:
# 2行目だけ残して後は削除
df_market13 = df_market11.drop(df_market11.index[[1,3,4]]).reset_index(drop=True)

In [48]:
# 3行目だけ残して後は削除
df_market14 = df_market11.drop(df_market11.index[[1,2,4]]).reset_index(drop=True)

In [49]:
df_market15 = pd.concat([df_market12, df_market13, df_market14], axis=1)

In [50]:
df_market15

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,日経225,TOPIX,JPX日経400,東証プライム市場指数,東証スタンダード市場指数,東証グロース市場指数,旧東証市場第一部指数,東証グロース市場Core指数,マザーズ総合指数,日経平均VI指数,...,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数
1,"26,847.90","1,899.62","17,122.20",977.08,965.34,878.94,"1,899.62",844.80,694.41,25.07,...,5.49%,1.53%,2.41%,2.40%,2.23%,2.43%,0.65%,-0.02%,0.20%,0.45%


In [51]:
# 行列入れ替え
df_market21 = df_market2.T

In [52]:
# 1行目だけ残して後は削除
df_market22 = df_market21.drop(df_market21.index[[2,3,4]]).reset_index(drop=True)

In [53]:
# 2行目だけ残して後は削除
df_market23 = df_market21.drop(df_market21.index[[1,3,4]]).reset_index(drop=True)

In [54]:
# 3行目だけ残して後は削除
df_market24 = df_market21.drop(df_market21.index[[1,2,4]]).reset_index(drop=True)

In [55]:
df_market25 = pd.concat([df_market22, df_market23, df_market24], axis=1)

In [56]:
df_market25

,1,2,3,4,5,6,7,8,9,10,...,9,10,11,12,13,14,15,16,17,18
0,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,...,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures
1,"27,210","26,730","26,755.00",702.0,"17,395","11,737.97","13,696.31","8,468.84","7,129.59",0,...,-0.42%,--%,--%,-3.64%,-2.72%,-2.63%,-4.48%,-2.80%,-2.02%,-0.97%


In [57]:
# 行列入れ替え
df_market31 = df_market3.T

In [58]:
# 1行目だけ残して後は削除
df_market32 = df_market31.drop(df_market31.index[[2,3,4,5]]).reset_index(drop=True)

In [59]:
# 2行目だけ残して後は削除
df_market33 = df_market31.drop(df_market31.index[[1,3,4,5]]).reset_index(drop=True)

In [60]:
# 3行目だけ残して後は削除
df_market34 = df_market31.drop(df_market31.index[[1,2,4,5]]).reset_index(drop=True)

In [61]:
# 4行目だけ残して後は削除
df_market35 = df_market31.drop(df_market31.index[[1,2,3,5]]).reset_index(drop=True)

In [62]:
df_market36 = pd.concat([df_market32, df_market33, df_market34, df_market35], axis=1)

In [63]:
df_market36

,1,2,3,4,5,6,7,8,9,10,...,11,12,13,14,15,16,17,18,19,20
0,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,...,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円
1,129.75,136.84,91.70,83.82,100.96,163.20,133.34,16.5456,8.7358,8.2052,...,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%


In [64]:
# 行列入れ替え
df_market41 = df_market4.T

In [65]:
# 1行目だけ残して後は削除
df_market42 = df_market41.drop(df_market41.index[[2,3,4]]).reset_index(drop=True)

In [66]:
# 2行目だけ残して後は削除
df_market43 = df_market41.drop(df_market41.index[[1,3,4]]).reset_index(drop=True)

In [67]:
# 3行目だけ残して後は削除
df_market44 = df_market41.drop(df_market41.index[[1,2,4]]).reset_index(drop=True)

In [68]:
df_market45 = pd.concat([df_market42, df_market43, df_market44], axis=1)

In [69]:
pd.set_option('display.max_rows', None) 
# defult:60,制限なし:None
pd.set_option('display.max_columns', None) 
# defult:20,制限なし:None
df_market45

,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10
0,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）
1,104.11,7.275,"1,896.3959","1,906",937.6,22.735,4.378,818.25,"1,044","1,713",-1.25,+0.385,+2.0536,+17.3,+11.7,-0.394,-0.0355,+2.25,-30,+6.5,-1.19%,5.59%,0.11%,0.92%,1.29%,-1.70%,-0.80%,0.28%,-2.79%,0.38%


In [70]:
# 行列入れ替え
df_market51 = df_market5.T

In [71]:
# df_market51 = df_market51.reset_index
df_market52 = df_market51.drop(df_market51.index[[2,3]]).reset_index(drop=True)

In [72]:
df_market53 = df_market51.drop(df_market51.index[[1,3]]).reset_index(drop=True)

In [73]:
df_market54 = pd.concat([df_market52, df_market53], axis=1)

In [74]:
df_market54

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債
1,-0.038,0.004,0.220,2.894,2.938,0.936,1.910,1.450,1.998,2.025,0.936,2.778,3.364,7.010,+0.003,0.000,+0.003,0.000,0.000,-0.002,+0.003,+0.002,+0.094,-0.007,-0.002,-0.002,+0.029,+0.170


In [75]:
df_market61 = df_market6.T

In [76]:
df_market62 = df_market61.drop(df_market61.index[2]).reset_index(drop=True)

In [77]:
df_market62

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,日本 無担保コール翌日物,日本 公定歩合,アメリカ フェデラルファンド金利,カナダ 翌日物金利誘導目標,ユーロ 市場調整金利,イギリス レポ金利,スイス 3ヵ月物LIBOR誘導目標,南アフリカ レポ金利,香港 再割引金利,オーストラリア 政策金利,ニュージーランド 目標短期金利,インド レポ金利,中国 貸出金利,ブラジル Selicターゲットレート,ロシア リファイナンス金利,トルコ 中央銀行ON DEPO,メキシコ オーバーナイト金利
1,-0.021,0.3,0.50,1.00,--,0.75,-0.7500,4.25,5,0.10,1.50,4.00,4.35,11.75,--,12.50,6.61


In [78]:
df_marketall = pd.concat([df_market15, df_market25, df_market36, df_market45, df_market54, df_market62], axis=1)

In [79]:
#行と列の数を取り出す
sh=df_marketall.shape
#shapeの列の数だけrangeで振り直してcolumnsに代入
df_marketall.columns=range(sh[1])

In [80]:
# ,inplace=True
df_marketall.drop(columns=df_marketall.columns[[3,4,5,6,7,34,35,36,37,38,
                                               65,66,67,68,69]],inplace=True)

In [81]:
df_marketall

,0,1,2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301
0,日経225,TOPIX,JPX日経400,マザーズ総合指数,日経平均VI指数,東証REIT指数,NYダウ,NASDAQ,S&P500指数,英FT100指数,仏CAC40指数,独DAX30指数,ロシアRTS指数,ブラジルボベスパ指数,インドSENSEX,香港ハンセン指数,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数,日経225,TOPIX,JPX日経400,マザーズ総合指数,日経平均VI指数,東証REIT指数,NYダウ,NASDAQ,S&P500指数,英FT100指数,仏CAC40指数,独DAX30指数,ロシアRTS指数,ブラジルボベスパ指数,インドSENSEX,香港ハンセン指数,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数,日経225,TOPIX,JPX日経400,マザーズ総合指数,日経平均VI指数,東証REIT指数,NYダウ,NASDAQ,S&P500指数,英FT100指数,仏CAC40指数,独DAX30指数,ロシアRTS指数,ブラジルボベスパ指数,インドSENSEX,香港ハンセン指数,ハンセンH株指数,ハンセンレッドチップ指数,上海総合指数,上海A株指数,上海・香港ストックコネクト,CSI300,シンガポールST指数,タイSET指数,クアラルンプール総合指数,ジャカルタ総合指数,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures,日経225先物（期近）,CME日経平均先物,SGX日経平均先物,東証マザース先物（期近）,JPX日経400先物（期近）,JPX国債先物,JPX国債先物レバレッジ,JPX国債先物インバース,JPX国債先物ダブルインバース,台湾加権先物（期近）,FTSE中国50先物（期近）,E-mini S&P 500,E-mini S&P Mid Cap 400,E-mini S&P Small Cap 600,E-mini NASDAQ-100,E-mini Dow,E-mini Nikkei225,SGX Nifty 50 Index Futures,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,米ドル/円,ユーロ/円,豪ドル/円,NZドル/円,カナダドル/円,英ポンド/円,スイスフラン/円,香港ドル/円,トルコリラ/円,南アランド/円,ロシアルーブル/円,韓国ウォン/円,メキシコペソ/円,ブラジルレアル/円,インドルピー/円,中国人民元/円,シンガポールドル/円,タイバーツ/円,マレーシアリンギット/円,インドネシアルピア/円,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,原油（WTI原油先物）,天然ガス（Henry Hub）先物（NYMEX）,金（現物 1oz.あたり）,Gold先物（COMEX),プラチナ先物（NYMEX）,Silver先物（COMEX),Copper先物（COMEX),Corn先物（CBOT）,Wheat先物（CBOT）,Soybeans先物（CBOT）,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債,日本国債3年,日本国債5年,日本国債10年,米国3年国債,米国10年国債,ドイツ10年国債,イギリス10年国債,フランス10年国債,スペイン10年国債,ポルトガル10年国債,ユーロ圏10年国債,イタリア10年国債,ギリシャ10年国債,ハンガリー10年国債,日本 無担保コール翌日物,日本 公定歩合

In [82]:
# プラウザを閉じる
driver.quit()

In [83]:
df_marketall.to_csv('data/market_' + filedate + '.csv')